In [5]:
import pymc3 as pm
from pymc3.glm import glm
import numpy as np
import pandas as pd
import datetime
import scipy.stats

from scipy import optimize
import theano as thno
import theano.tensor as T

# data retrieval helper module
from general.DB import DB
import util

from sklearn.linear_model import LinearRegression

# plotting libraries
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import seaborn as sns
%matplotlib inline

In [6]:
# get the data we need from the database
year = 2015
games_df, stacked, teams = util.get_data(year)
games_df = games_df.sort('dt')  # sort by date ascending
kenpom = pd.read_sql("SELECT team, adjt FROM kenpom_ranks WHERE year = %s" % year, DB.conn)
all_teams = pd.read_sql("SELECT ncaa, ncaaid, kenpom FROM teams", DB.conn)
teams = teams.merge(all_teams, left_on='team_id', right_on='ncaaid')
teams = teams.merge(kenpom, how='left', left_on='kenpom', right_on='team').drop(['team', 'kenpom', 'ncaaid'], 1)
num_teams = teams.shape[0]
print("Got data for %s games and %s teams, between %s and %s" % (games_df.shape[0], num_teams,
                                                   datetime.datetime.strftime(games_df['dt'].min(), "%m-%d-%Y"),
                                                  datetime.datetime.strftime(games_df['dt'].max(), "%m-%d-%Y")))

Got data for 5279 games and 351 teams, between 11-16-2014 and 04-06-2015


In [7]:
stacked.shape

(10558, 20)

In [8]:
def get_indices(unstacked, approx_burn_games, approx_interval):
    date_counts = unstacked.groupby('dt').count()['game_id']
    cum_indices = np.cumsum(date_counts).values
    next_ = cum_indices[-1]
    indices = []
    for gp in cum_indices[::-1]:
        if gp <= approx_burn_games:
            break
        if gp <= next_:
            indices.append(gp)
            next_ -= approx_interval
    return indices[::-1]

def get_points_matrix(home_idx, away_idx, home_ppp, away_ppp, is_neutral, is_log=False):
    home_off_mat = pd.get_dummies(home_idx, prefix='o')
    away_def_mat = pd.get_dummies(away_idx, prefix='d')
    df1 = pd.concat([home_off_mat, away_def_mat], axis=1)
    df1['ppp'] = home_ppp if not is_log else np.log(home_ppp)
    df1['home'] = (~is_neutral).astype(int)

    home_def_mat = pd.get_dummies(home_idx, prefix='d')
    away_off_mat = pd.get_dummies(away_idx, prefix='o')
    df2 = pd.concat([away_off_mat, home_def_mat], axis=1)
    df2['ppp'] = away_ppp if not is_log else np.log(away_ppp)
    df2['home'] = is_neutral.astype(int) - 1.0

    df = pd.concat([df1, df2], axis=0).sort_index()
    predictors = ['o_%s' % i for i in range(num_teams - 1)] + ['d_%s' % i for i in range(num_teams - 1)] + ['home']
    X = df[predictors].values
    y = df['ppp'].values
    return X, y

def extract_coef(lr, num_teams):
    o = np.zeros(num_teams)
    o[:-1] = lr.coef_[:num_teams - 1]
    d = np.zeros(num_teams)
    d[:-1] = lr.coef_[num_teams - 1:2 * num_teams - 2]
    return o, d

In [9]:
def get_temporal_weights(N, weight_type="exponential"):
    weights = np.ones(2*N)
    if weight_type == "exponential":
        memory = 1000
        if N < memory:
            return weights
        else:
            idx = np.arange(N - memory)*2
            weights[idx] = np.exp(np.arange(-(N - memory), 0) / 500.)
            weights[idx + 1] = np.exp(np.arange(-(N - memory), 0) / 500.)
    elif weight_type == "window":
        window_len = 3000
        if N < window_len:
            return weights
        else:
            weights[:window_len*2] = 0
    else:
        pass
    
    return weights

def get_mov_weights(pred_diffs, diffs):
    mov = np.array([mov_weights(p, d) for p, d in zip(pred_diffs, diffs)])
    weights = np.zeros(pred_diffs.shape[0] *2)
    idx = np.arange(pred_diffs.shape[0]) * 2
    weights[idx] = mov
    weights[idx + 1] = mov
    return weights

def get_poss_weights(poss, N):
    weights = np.zeros(2 * N)
    idx = np.arange(N) * 2
    weights[idx] = 1 / poss[:N]
    weights[idx + 1] = 1 / poss[:N]
    return weights

def mov_weights(pred_diff, diff):
    if pred_diff > 0.2 and diff > 0.2:
        # home expected big win and was big win
        weight = 0.2
    elif pred_diff > 0.2 and diff < 0:
        # home expected big win and was loss
        weight = 2
    elif pred_diff < -0.2 and diff < -0.2:
        # away win big and away won big
        weight = 0.2
    elif pred_diff < -0.2 and diff > 0:
        # away expected win big and lost
        weight = 2
    else:
        weight = 1
    return weight
    
def rate_for_season(season_data, skip_interval=100, temporal_weights="all", is_log=False, verbose=False):
    X, y = get_points_matrix(season_data['i_hteam'].values, season_data['i_ateam'].values, season_data['hppp'].values, 
        season_data['appp'].values, season_data['neutral'].values, is_log=is_log)
    
    # fit all the data initially to get the home_factor
    zero_lr = LinearRegression()
    zero_lr.fit(X, y)
    home_factor = zero_lr.coef_[-1]
    if verbose:
        print("Home factor for %s season is %1.3f points per possession" % (year, home_factor))
    home_adjustment = X[:, -1] * home_factor * -1

    all_home_indices = season_data['i_hteam'].values
    all_away_indices = season_data['i_ateam'].values
    all_home_off_ratings = np.ones(season_data.shape[0]) * -1
    all_home_def_ratings = np.ones(season_data.shape[0]) * -1
    all_away_off_ratings = np.ones(season_data.shape[0]) * -1
    all_away_def_ratings = np.ones(season_data.shape[0]) * -1
    all_intercepts = np.ones(season_data.shape[0]) * -1
    
    indices = get_indices(games_df, 1000, skip_interval)
    prev_idx = indices[0]
    _lr = LinearRegression()
    for i, idx in enumerate(indices[1:]):
        hppp_pred = all_home_off_ratings[:prev_idx] + all_away_def_ratings[:prev_idx] + home_factor + \
            all_intercepts[:prev_idx]
        appp_pred = all_away_off_ratings[:prev_idx] + all_home_def_ratings[:prev_idx] - home_factor + \
            all_intercepts[:prev_idx]
        ppp_diff_pred = hppp_pred - appp_pred
        ppp_diff = (season_data['hppp'] - season_data['appp']).values[:prev_idx]
        mov_weights = get_mov_weights(ppp_diff_pred, ppp_diff)
        _X = X[:prev_idx * 2, :-1]
        _y = y[:prev_idx * 2] + home_adjustment[:prev_idx * 2]
        _lr.fit(_X, _y, sample_weight=get_poss_weights(season_data['poss'].values, prev_idx))
        if i % 10 == 0 and verbose:
            print("Iteration %s: fit linear regression on %s rows and %s columns" % (i, _X.shape[0], _X.shape[1]))
        ortg, drtg = extract_coef(_lr, num_teams)

        _home_indices = all_home_indices[prev_idx:idx]
        _away_indices = all_away_indices[prev_idx:idx]
        all_home_off_ratings[prev_idx:idx] = ortg[_home_indices]
        all_away_off_ratings[prev_idx:idx] = ortg[_away_indices]
        all_home_def_ratings[prev_idx:idx] = drtg[_home_indices]
        all_away_def_ratings[prev_idx:idx] = drtg[_away_indices]
        all_intercepts[prev_idx:idx] = _lr.intercept_
        prev_idx = idx
    return {'hortg': all_home_off_ratings, 'hdrtg': all_home_def_ratings,
            'aortg': all_away_off_ratings, 'adrtg': all_away_def_ratings, 
            'intercept': all_intercepts}, home_factor

In [10]:
def add_ratings(df, ratings, home_factor):
    for col, rtg in ratings.items():
        df[col] = rtg
    df['month'] = df['dt'].map(lambda dt: dt.month)
    df['hpred'] = (df['hortg'] + df['adrtg'] + home_factor * ~df['neutral'] + df['intercept']).values
    df['apred'] = (df['aortg'] + df['hdrtg'] - home_factor * ~df['neutral'] + df['intercept']).values
    df['outcome_pred'] = df['hpred'] > df['apred']
    df['correct'] = df['outcome_pred'] == df['home_outcome']
    return df[df['hortg'] != -1.]

In [11]:
relevant_cols = ['dt', 'hteam', 'ateam', 'hppp', 'appp', 'neutral', 'home_outcome']
ratings, home_factor = rate_for_season(games_df, skip_interval=100, temporal_weights="all", is_log=False)
df0 = add_ratings(games_df[relevant_cols].copy(), ratings, home_factor)

/usr/local/lib/python3.4/site-packages/scipy/linalg/basic.py:884: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


In [12]:
games_df.head()

,game_id,dt,hteam,hteam_id,ateam,ateam_id,hpts,hposs,apts,aposs,i_hteam,i_ateam,hppp,appp,neutral,home_outcome,poss,season
4030,3509779,2014-11-16,Miami (OH),414.0,Southern Utah,667.0,76,71.150,63,67.500,156,268,1.068166,0.933333,False,True,69.3250,2015
2617,3512375,2014-11-16,East Carolina,196.0,UNC Asheville,456.0,79,77.250,83,76.150,77,173,1.022654,1.089954,False,False,76.7000,2015
3375,3512373,2014-11-16,UCF,128.0,Stetson,678.0,64,62.825,55,64.125,49,274,1.018703,0.857700,False,True,63.4750,2015
3419,3510544,2014-11-16,Seton Hall,635.0,Mercer,406.0,63,65.025,47,63.750,252,155,0.968858,0.737255,False,True,64.3875,2015
847,3512441,2014-11-16,Virginia,746.0,Norfolk St.,485.0,67,58.075,39,56.275,308,189,1.153681,0.693025,False,True,57.1750,2015


In [18]:
print(df0[df0['month'] != 11]['correct'].mean())
pd.merge(df0.groupby('month').mean()[['correct']], df0.groupby('month').count()[['correct']], 
         left_index=True, right_index=True)

0.715472235406


,correct_x,correct_y
month,,
1,0.717812,1499
2,0.703331,1291
3,0.717156,647
4,0.666667,6
12,0.730220,771


In [153]:
ratings, home_factor = rate_for_season(games_df, skip_interval=300, verbose=True)
df1 = add_ratings(games_df[relevant_cols].copy(), ratings, home_factor)

Home factor for 2016 season is 0.023 points per possession
Iteration 0: fit linear regression on 2474 rows and 700 columns
Iteration 10: fit linear regression on 8470 rows and 700 columns


In [131]:
ratings, home_factor = rate_for_season(games_df, skip_interval=300, is_log=True, temporal_weights=True, verbose=True)
df2 = add_ratings(games_df[relevant_cols].copy(), ratings, home_factor)

Home factor for 2016 season is 0.022 points per possession
Iteration 0: fit linear regression on 2474 rows and 700 columns
Iteration 10: fit linear regression on 8470 rows and 700 columns


In [130]:
ratings, home_factor = rate_for_season(games_df, skip_interval=300, temporal_weights=True, verbose=True)
df3 = add_ratings(games_df[relevant_cols].copy(), ratings, home_factor)

Home factor for 2016 season is 0.023 points per possession
Iteration 0: fit linear regression on 2474 rows and 700 columns
Iteration 10: fit linear regression on 8470 rows and 700 columns


In [134]:
print(df1['correct'].mean())
pd.merge(df1.groupby('month').mean()[['correct']], df1.groupby('month').count()[['correct']], 
         left_index=True, right_index=True)

0.711457378552


,correct_x,correct_y
month,,
1,0.699243,1453
2,0.707317,1353
3,0.717460,630
4,0.500000,4
11,0.686084,927
12,0.751838,1088


In [138]:
df0.shape

(5455, 17)

In [121]:
kp = pd.read_sql("SELECT * FROM kp", DB.conn)

In [281]:
kp['month'] = kp['dt'].map(lambda dt: dt.month)
kp['season'] = kp['dt'].map(lambda dt: dt.year if dt.month < 6 else dt.year + 1)
kp['outcome'] = kp['score1'] > kp['score2']
kp['pred_outcome'] = kp['predicted_score'] > kp['predicted_score2']
kp['mov'] = kp['score1'] - kp['score2']
kp['correct'] = kp['team1'] == kp['predicted']
kp = kp[kp['month'] != 11]
kp_gb = kp.groupby(['month', 'season']).mean()[['correct', 'mov']]

In [282]:
print(kp[kp['season'] == 2015]['correct'].mean())
kp_gb

0.722027972028


correct        mov
month season                     
1     2011    0.721831  11.194366
      2012    0.726096  11.148814
      2013    0.712446  11.196710
      2014    0.683140  10.797238
      2015    0.714286  10.457672
2     2011    0.716144  10.319816
      2012    0.749451  10.653480
      2013    0.715962  11.008607
      2014    0.735994  10.425173
      2015    0.703280  10.756674
3     2011    0.717428  10.807107
      2012    0.718147  10.415058
      2013    0.712975  10.560944
      2014    0.716456  10.554430
      2015    0.718894   9.927803
4     2011    0.750000   5.750000
      2012    1.000000   8.000000
      2013    0.700000   7.300000
      2014    0.363636   9.000000
      2015    0.500000   6.833333
12    2011    0.803366  13.007972
      2012    0.793345  13.385289
      2013    0.797862  13.693878
      2014    0.767196  13.041270
      2015    0.758212  12.989964

In [129]:
sub = games_df[~games_df['neutral']].copy()
sub['month'] = sub['dt'].map(lambda d: d.month)
sub.groupby('month').mean()[['home_outcome']]

,home_outcome
month,
1,0.585114
2,0.623521
3,0.631922
4,0.500000
11,0.765130
12,0.715407


In [218]:
kp

,dt,team1,score1,team2,score2,predicted,predicted_score,predicted_score2,venue,city,state,tm,month,outcome,pred_outcome,correct
0,2010-11-08,Pittsburgh,83,Rhode Island,75,Pittsburgh,78,61,Petersen Events Center,Pittsburgh,PA,TBA,11,True,True,True
1,2010-11-08,Illinois,79,UC Irvine,65,Illinois,77,57,Assembly Hall,Champaign,IL,TBA,11,True,True,True
2,2010-11-08,Maryland,105,Seattle,76,Maryland,91,66,Comcast Center,College Park,MD,TBA,11,True,True,True
3,2010-11-08,Texas,83,Navy,52,Texas,87,60,Frank Erwin Special Events Cen,Austin,TX,TBA,11,True,True,True
4,2010-11-10,Maryland,75,College of Charleston,74,Maryland,86,70,Comcast Center,College Park,MD,TBA,11,True,True,True
5,2010-11-10,Pittsburgh,97,Illinois Chicago,54,Pittsburgh,82,59,Petersen Events Center,Pittsburgh,PA,TBA,11,True,True,True
6,2010-11-10,Texas,89,Louisiana Tech,58,Texas,84,59,Frank Erwin Special Events Cen,Austin,TX,TBA,11,True,True,True
7,2010-11-10,Illinois,84,Toledo,45,Illinois,81,49,Assembly Hall,Champaign,IL,TBA,11,True,True,True
8,2010-11-12,Georgetown,62,Old Dominion,59,Georgetown,64,63,Constant Convocation Center,Norfolk,VA,TBA,11,True,True,True
9,2010-11-12,Temple,62,Seton Hall,56,Temple,71,65,Liacouras Center,Philadelphia,PA,TBA,11,True,True,True


In [208]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score

lr = LogisticRegression()
rf = RandomForestClassifier(max_depth=5)
X = df0[['hortg', 'aortg', 'hdrtg', 'adrtg']].values
y = df0['home_outcome'].values
cvs = cross_val_score(lr, X, y, cv=10)

In [209]:
cvs, np.mean(cvs)

(array([ 0.72813239,  0.78486998,  0.70685579,  0.69030733,  0.68646081,
         0.71971496,  0.6888361 ,  0.70783848,  0.72209026,  0.71021378]),
 0.71453198789328565)